In [ ]:
!pip install bs4

In [ ]:
!pip install requests

In [ ]:
import bs4
import requests
import csv
import string
import nltk
import numpy as np
!pip install Unidecode
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer


     |████████████████████████████████| 245kB 3.3MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


We'll be starting the collection of our corpus from Wikipedia webpage. To find the medical data we can find numerous links and words on the word 'Medical' on wikipedia. 

In [ ]:
req = requests.get('https://en.wikipedia.org/wiki/Medicine') 
soup = bs4.BeautifulSoup(req.text,'lxml')

In [ ]:
# this code generates a list with all the webscrapped links
count = 0
linklist = [all]
for link in soup.find_all('a',href = True):
  if link['href'][0] == '/':
    link['href'] = 'https://en.wikipedia.org/'+link['href']
  if link['href'][0] != '#':
    #print(link['href'])
    count+=1
    linklist.append(link['href'])

In [ ]:
print('The total number of sites are:',count) #on webscrapping we found out 1373 sites related to Medical

The total number of sites are: 1373


In [ ]:
len(linklist)

1374

In [ ]:
# code for preprocessing and cleaning the text, we'll be using it later for getting the output medical words
def preprocessing(text):
  text = unidecode(text)
  engwords = set(nltk.corpus.words.words())
  text = " ".join(w for w in nltk.wordpunct_tokenize(text) \
         if w.lower() in engwords or not w.isalpha())
  text = text.replace("\n"," ")
  text = text.replace("\t"," ")
  punctuations = '''!()-[]{};:'"\,<>|./?@#$%^&*_~1234567890'''
  for x in text:
    if x in punctuations: 
      text = text.replace(x, " ")
  corpus = text.lower().split(" ")
  stop_words = stopwords.words('english')+list(string.punctuation)
  cleantext = []
  for i in corpus:
    i = i.replace("\n"," ")
    for x in i: 
        if x in punctuations: 
            i = i.replace(x, "") 
    cleantext.append(i)
  morecleantext = []
  lem = WordNetLemmatizer()
  for w in cleantext:
    if w not in stop_words:
      w = lem.lemmatize(w)
      morecleantext.append(w)
  morecleantext = list(dict.fromkeys(morecleantext))
  morecleantext = ' '.join(morecleantext)
  return morecleantext

In [ ]:
# creating a text document with all the medical data from the links which is then preprocessed from the above code.
with open('data.txt', 'w') as f:
  for i in linklist:
    try:
      req = requests.get(i)
      soup = bs4.BeautifulSoup(req.text,'lxml')
      [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
      visible_text = soup.getText()
      soup = preprocessing(visible_text)
      f.writelines(soup)
    except:
      pass

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#this code gives us a mapping of the word and the word vector
def read_glove_vecs(glove_file):
      with open(glove_file, 'r') as f:
          words = set()         # ensures unique values
          word_to_vec_map = {}  # this will be a dictionary mapping words to their vectors
          for line in f:
              line = line.strip().split()
              curr_word = line[0]
              words.add(curr_word)
              word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
      return word_to_vec_map,words
word_to_vec_map,words = read_glove_vecs('/content/drive/My Drive/Copy of glove.6B.50d.txt')

In [ ]:
with open('data.txt','r') as f:
  text = f.read()
  text = text.lower().split(" ")
  text = list(dict.fromkeys(text))
  with open('/content/drive/My Drive/in_vocab1.txt','a+') as f:
    for i in text:
      if i in words:
        f.writelines(i)
        f.writelines(' ')
  with open('/content/drive/My Drive/out_vocab1.txt','a+') as g:
    for i in text:
      if i not in words:
        g.writelines(i) 
        g.writelines(' ')
print(len(text))

40442


In [ ]:
# saving words which are out of vocabulary in a seperate file  (words not in glove embedding)
with open('/content/drive/My Drive/out_vocab1.txt','r') as g:
  outvocab = g.read()
  outvocab = outvocab.lower().split(" ")
#outvocab

In [ ]:
# saving words which are in vocab in a seperate file (the medical words which are in glove embedding)
with open('/content/drive/My Drive/in_vocab1.txt','r') as f:
  invocab = f.read()
  invocab = invocab.lower().split(" ")
#invocab